## Goal Prediction
### 1. Load Vectorstore
- Manage goal predicition module separately

In [1]:
import openai
import langchain
import logging
import json
import os
import pandas as pd
import pickle
from IPython.display import Image, display


In [7]:
# Embedding Database
import database
from langchain_openai.embeddings import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import sys
sys.path.append(os.path.abspath('/usr/local/lib/python3.10/dist-packages'))

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# configure paths
data_path = '../data/ego4d_annotation/'
goalstep_annotation_path = data_path + 'goalstep/'
spatial_annotation_path = data_path + 'spatial/'

# extract videos list
goalstep_videos_list = database.merge_json_video_list(goalstep_annotation_path)
spatial_videos_list = database.merge_json_video_list(spatial_annotation_path)
print(f"goalstep vids: {len(goalstep_videos_list)} and spatial vids: {len(spatial_videos_list)}")

# make document list from the video list
goalstep_document_list = database.make_goalstep_document_list(goalstep_videos_list)
spatial_document = database.make_spatial_document_list(spatial_videos_list)
print(f"goalstep_document_list: {len(goalstep_document_list)}")
print(f"spatial_document_list: {len(spatial_document)}")

# make vectorstores for each dataset (takes most time-3min)
embeddings = OpenAIEmbeddings()
goalstep_vector_store =  DocArrayInMemorySearch.from_documents(goalstep_document_list, embedding=embeddings)
spatial_vector_store = DocArrayInMemorySearch.from_documents(spatial_document, embedding=embeddings)


goalstep vids: 717 and spatial vids: 36
goalstep_document_list: 39979
spatial_document_list: 400


/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


### Retrieve with Query made with Inputs

In [ ]:
print(os.getcwd())
import sys
import input_source
sys.path.append(os.path.abspath('/root/project')) # add root path to sys.path for external package
from util import metrics

from langchain.vectorstores import FAISS, Chroma
from langchain.schema import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever

# can import other scripts in other folders fine!
metrics.printtest()

# Input from Source Space (context & lv2 actions)
input_video_idx = 0
goalstep_video = goalstep_videos_list[input_video_idx]
spatial_video = spatial_videos_list[input_video_idx]
input_goalstep_segments = input_source.extract_lev2_goalstep_segments(goalstep_video)
input_spatial_context = input_source.extract_spatial_context(spatial_video)

# Create retrievers for both vectorstores
goalstep_retriever = goalstep_vector_store.as_retriever()
spatial_retriever = spatial_vector_store.as_retriever()

# do not use multiretriever since there are too much errors!
query = f"Here is the {input_goalstep_segments} and {input_spatial_context}"
retrieved_stuff1 = goalstep_retriever.get_relevant_documents(query)
retrieved_stuff2 = spatial_retriever.get_relevant_documents(query)
print(retrieved_stuff1)
print(retrieved_stuff2)

# Retrieve documents' parent documents for goalstep annotation

/root/project/script_predict_goal
metrics.pssSy
[Document(metadata={'type': 'level2', 'video_uid': '543e4c99-5d9f-407d-be75-c397d633fe56', 'start_time': 0.62112, 'end_time': 15.483179999999999, 'step_category': 'Make recipes: Take two slices of bread and put them on a plate', 'step_description': 'Place two slices on bread on the plate'}, page_content='Level 2 Segment 1 for Video 543e4c99-5d9f-407d-be75-c397d633fe56\nStep: Place two slices on bread on the plate'), Document(metadata={'type': 'level3', 'video_uid': '7921902b-293f-49e2-9401-d96791a90e15', 'parent_level1_start_time': 137.46841, 'start_time': 142.81772, 'end_time': 188.65649, 'step_category': 'Make recipes: Take two slices of bread and put them on a plate', 'step_description': 'get two pieces of  sliced bread '}, page_content='Level 3 Segment 1 for Level 2 Segment 3 in Video 7921902b-293f-49e2-9401-d96791a90e15\nStep: get two pieces of  sliced bread '), Document(metadata={'type': 'level2', 'video_uid': '94d5eff8-0fac-4719-ad

### Configure Prompt and Invoke LLM

In [6]:
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI

import prompt_source as promptSource
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate


# define llm
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=openai.api_key, model="gpt-3.5-turbo")

# define prompt
prompt = ChatPromptTemplate.from_template(promptSource.template_source)
prompt.format(context=promptSource.context, question=promptSource.question, rules = promptSource.rules)

# Define chain
parser = StrOutputParser()
chain1 = prompt | model | parser 

# Get Respone
response = chain1.invoke()


### Process Response and Compute Distance

In [ ]:
# postprocess answers to get goals



# postprocess answers to get lv1 steps?



In [ ]:
# compute metrics



### Configure Prompt2 for Target Space

In [ ]:
import input_target
import prompt_target

# target space input
input_target_goalstep_segments = input_target.extract_lev2_goalstep_segments()
input_target_spatial_context = input_target.extract_spatial_context()

# make target query


# retrieve with target query


# make prompt with query and target inptu


# make chain & invoke



### Configure Combined chain for simple one-go-prediction

In [ ]:

# # chain can incorpoate other chains
# chain_action = (
#     {"source_spatial_context": itemgetter(source_spatial_context), "goalstep": chain1, "target_spatial_context": itemgetter(target_spatial_context)} | prompt_action | model | parser
# )



from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI

import prompt_source as promptSource
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate


# define llm
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=openai.api_key, model="gpt-3.5-turbo")

# define prompt
prompt = ChatPromptTemplate.from_template(promptSource.template_source)
prompt.format(context=promptSource.context, question=promptSource.question, rules = promptSource.rules)

# Define chain
parser = StrOutputParser()
chain1 = prompt | model | parser 

# Get Respone
response = chain1.invoke()